In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from itertools import permutations
import time
import imageio
import os

In [2]:
data = {}
data["distance_matrix"] = [
        [0,    2451, 2408, 213,  2571, 1972],  
        [2451, 0,    959,  2596, 403,  579],   
        [2408, 959,  0,    2493, 678,  701],   
        [213,  2596, 2493, 0,    2699, 2099], 
        [2571, 403,  678,  2699, 0,    600],   
        [1972, 579,  701,  2099, 600,  0],    
    ]

In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Coordinates of the cities (latitude, longitude)
    data["cities_info"] = {
        "New York": (40.7128, -74.0060),
        "Los Angeles": (34.0522, -118.2437),
        "Chicago": (41.8781, -87.6298),
        "Denver": (39.7392, -104.9903),
        "Dallas": (32.7767, -96.7970),
        "Seattle": (47.6062, -122.3321),
        "Boston": (42.3601, -71.0589),
        "San Francisco": (37.7749, -122.4194),
        "St. Louis": (38.6270, -90.1994),
        "Houston": (29.7604, -95.3698),
        "Phoenix": (33.4484, -112.0740),
        "Salt Lake City": (40.7608, -111.8910),
    }
    # List of cities in the order corresponding to the distance matrix
    data["cities"] = [
        "New York",
        "Los Angeles",
        "Chicago",
        "Denver",
        "Dallas",
        "Seattle",
        "Boston",
        "San Francisco",
        "St. Louis",
        "Houston",
        "Phoenix",
        "Salt Lake City",
    ]
    # Distance matrix in miles
    data["distance_matrix"] = [
        # NY    LA    Chicago Denver Dallas Seattle Boston San Francisco St. Louis Houston Phoenix Salt Lake
        [0,    2451, 713,    1631, 1374, 2408, 213,    2571, 875,    1420, 2145, 1972],  # New York
        [2451, 0,    1745,   831, 1240, 959,  2596, 403,   1589, 1374, 357, 579],      # Los Angeles
        [713,  1745, 0,      920, 803, 1737, 851,  1858, 262,    940, 1453, 1260],      # Chicago
        [1631, 831,  920,    0,   663, 1021, 1769, 949,   796,    879, 586, 371],       # Denver
        [1374, 1240, 803,    663, 0,   1681, 1551, 1765, 547,    225, 887, 999],       # Dallas
        [2408, 959,  1737,  1021, 1681, 0,    2493, 678,   1724, 1891, 1114, 701],      # Seattle
        [213,  2596, 851,    1769, 1551, 2493, 0,    2699, 1038, 1605, 2300, 2099],    # Boston
        [2571, 403,  1858,   949, 1765, 678,  2699, 0,    1744, 1645, 653, 600],       # San Francisco
        [875,  1589, 262,    796, 547, 1724, 1038, 1744, 0,      679, 1272, 1162],      # St. Louis
        [1420, 1374, 940,    879, 225, 1891, 1605, 1645, 679,    0,    1017, 1200],     # Houston
        [2145, 357,  1453,   586, 887, 1114, 2300, 653,   1272, 1017, 0, 504],        # Phoenix
        [1972, 579,  1260,   371, 999, 701,  2099, 600,   1162, 1200, 504, 0],        # Salt Lake City
    ]
    data["num_cities"] = len(data["distance_matrix"])
    data["depot"] = 0  # Starting city (New York)
    return data

In [4]:
def calculate_total_distance(route, distance_matrix):
    """Calculates the total distance of the given route."""
    total_distance = 0
    for i in range(len(route) - 1):
        from_city = route[i]
        to_city = route[i + 1]
        total_distance += distance_matrix[from_city][to_city]
    return total_distance

In [5]:
def solve_tsp_brute_force_with_history(data, temp_dir):
    """Solves the TSP problem using brute force and records route history."""
    from itertools import permutations

    cities = data["cities"]
    distance_matrix = data["distance_matrix"]
    depot = data["depot"]
    num_cities = data["num_cities"]

    # Generate all possible routes starting and ending at the depot
    city_indices = list(range(num_cities))
    city_indices.remove(depot)  # Fix the depot at the start

    min_distance = float('inf')
    optimal_route = None
    route_history = []  # To store each new optimal route

    # Calculate total permutations: (n-1)!
    total_permutations = 1
    for i in range(2, num_cities):
        total_permutations *= i

    print(f"Total permutations to evaluate: {total_permutations}")
    start_time = time.time()

    permutation_count = 0

    for perm in permutations(city_indices):
        # Construct the full route starting and ending at the depot
        route = [depot] + list(perm) + [depot]
        # Calculate the total distance
        total_distance = 0
        valid = True
        for i in range(len(route) - 1):
            from_city = route[i]
            to_city = route[i + 1]
            distance = distance_matrix[from_city][to_city]
            total_distance += distance
            # Branch and Bound: if current distance exceeds min_distance, stop evaluating this permutation
            if total_distance >= min_distance:
                valid = False
                break
        if valid and total_distance < min_distance:
            min_distance = total_distance
            optimal_route = route
            # Convert route indices to city names
            route_cities = [data["cities"][city] for city in optimal_route]
            # Save the new optimal route to history
            route_history.append(list(route_cities))
            # Save the plot frame
            plot_route(list(route_cities), data["cities_info"], temp_dir, len(route_history))
            # Print progress
            elapsed = time.time() - start_time
            print(f"New optimal distance: {min_distance} miles found at permutation {permutation_count}")
            print(f"Route: {' -> '.join(route_cities)}")
        permutation_count += 1
        # Progress indicator every 1,000,000 permutations
        if permutation_count % 1000000 == 0:
            elapsed = time.time() - start_time
            print(f"Checked {permutation_count} permutations in {elapsed:.2f} seconds...")

    end_time = time.time()
    elapsed_time = end_time - start_time

    if optimal_route:
        return optimal_route, min_distance, permutation_count, elapsed_time, route_history
    else:
        return None, None, permutation_count, elapsed_time, route_history

In [6]:
def plot_route(route, cities_info, temp_dir, frame_number):
    """Plots the route and saves the plot as an image in the temporary directory."""
    # Extract latitude and longitude for the route
    lats = []
    lons = []
    for city in route:
        lat, lon = cities_info[city]
        lats.append(lat)
        lons.append(lon)

    # Create a plot with Cartopy
    plt.figure(figsize=(15, 10))
    ax = plt.axes(projection=ccrs.LambertConformal())
    ax.set_extent([-125, -66.5, 24, 50], ccrs.Geodetic())

    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.STATES, linestyle=':')

    # Plot the cities
    ax.scatter(lons, lats, color='red', s=100, transform=ccrs.Geodetic())

    # Annotate city names
    for city in route:
        lat, lon = cities_info[city]
        ax.text(lon + 0.5, lat + 0.5, city, fontsize=9, transform=ccrs.Geodetic())

    # Plot the route lines
    ax.plot(lons, lats, color='blue', linewidth=2, marker='o', transform=ccrs.Geodetic())

    # Add frame number as subtitle
    plt.title("Optimal TSP Route Across US Cities (Brute Force)", fontsize=16)
    plt.suptitle(f"Frame {frame_number}", fontsize=10, y=0.95)

    # Save the figure
    frame_filename = os.path.join(temp_dir, f"frame_{frame_number:05d}.png")
    plt.savefig(frame_filename)
    plt.close()

In [7]:
def create_gif(route_history, temp_dir, gif_filename="tsp_route.gif", duration=0.5):
    """Creates a GIF from the saved plot frames."""
    images = []
    print("Creating GIF...")
    for idx, route in enumerate(route_history):
        frame_filename = os.path.join(temp_dir, f"frame_{idx + 1:05d}.png")
        images.append(imageio.imread(frame_filename))
    imageio.mimsave(gif_filename, images, duration=duration)
    print(f"GIF saved as {gif_filename}")

In [8]:
def plot_route_final(route, cities_info):
    """Plots the final optimal route on a US map."""
    # Extract latitude and longitude for the route
    lats = []
    lons = []
    for city in route:
        lat, lon = cities_info[city]
        lats.append(lat)
        lons.append(lon)

    # Create a plot with Cartopy
    plt.figure(figsize=(15, 10))
    ax = plt.axes(projection=ccrs.LambertConformal())
    ax.set_extent([-125, -66.5, 24, 50], ccrs.Geodetic())

    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.STATES, linestyle=':')

    # Plot the cities
    ax.scatter(lons, lats, color='red', s=100, transform=ccrs.Geodetic())

    # Annotate city names
    for city in route:
        lat, lon = cities_info[city]
        ax.text(lon + 0.5, lat + 0.5, city, fontsize=9, transform=ccrs.Geodetic())

    # Plot the route lines
    ax.plot(lons, lats, color='blue', linewidth=2, marker='o', transform=ccrs.Geodetic())

    plt.title("Final Optimal TSP Route Across US Cities (Brute Force)", fontsize=16)
    plt.show()

In [ ]:
def main():
    # Create the data model
    data = create_data_model()

    # Create a temporary directory to store plot images
    temp_dir = "tsp_temp_frames"
    os.makedirs(temp_dir, exist_ok=True)

    print("Starting Brute Force TSP Solver...")
    print("This may take a considerable amount of time for 12 cities.")

    # Solve the TSP using brute force with route history
    route, total_distance, permutations_checked, elapsed_time, route_history = solve_tsp_brute_force_with_history(data, temp_dir)

    if route:
        # Convert city indices back to names
        route_cities = [data["cities"][city] for city in route]
        print("\nOptimal Route Found:")
        print(" -> ".join(route_cities))
        print(f"Total Distance: {total_distance} miles")
        print(f"Permutations Checked: {permutations_checked}")
        print(f"Elapsed Time: {elapsed_time:.2f} seconds")

        # Create the GIF from route history
        create_gif(route_history, temp_dir, gif_filename="tsp_route.gif", duration=0.5)
    else:
        print("No solution found!")
        print(f"Permutations Checked: {permutation_count}")
        print(f"Elapsed Time: {elapsed_time:.2f} seconds")

if __name__ == "__main__":
    main()

Starting Brute Force TSP Solver...
This may take a considerable amount of time for 12 cities.
Total permutations to evaluate: 39916800
New optimal distance: 18568 miles found at permutation 0
Route: New York -> Los Angeles -> Chicago -> Denver -> Dallas -> Seattle -> Boston -> San Francisco -> St. Louis -> Houston -> Phoenix -> Salt Lake City -> New York
New optimal distance: 18499 miles found at permutation 5
Route: New York -> Los Angeles -> Chicago -> Denver -> Dallas -> Seattle -> Boston -> San Francisco -> St. Louis -> Salt Lake City -> Phoenix -> Houston -> New York
New optimal distance: 17855 miles found at permutation 9
Route: New York -> Los Angeles -> Chicago -> Denver -> Dallas -> Seattle -> Boston -> San Francisco -> Houston -> Phoenix -> Salt Lake City -> St. Louis -> New York
New optimal distance: 17559 miles found at permutation 15
Route: New York -> Los Angeles -> Chicago -> Denver -> Dallas -> Seattle -> Boston -> San Francisco -> Phoenix -> Houston -> Salt Lake City -

C:\Users\henry\AppData\Local\Temp\ipykernel_8836\3353959196.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(frame_filename))


GIF saved as tsp_route.gif


In [8]:
import numpy as np
np.math.factorial(11)

C:\Users\henry\AppData\Local\Temp\ipykernel_27840\2747204335.py:2: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  np.math.factorial(11)


39916800